In [1]:
import os

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini-2024-07-18"

In [2]:
from pydantic import BaseModel
from typing import List

class SubSection(BaseModel):
    title:str
    content: str

class BlogPost(BaseModel):
    title:str
    introduction: str
    sections : List[SubSection]
    sources: List[str]
    hashtags: List[str]

In [3]:
from crewai import Crew, Agent, Task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information.",
    backstory="""
    You produce the highest quality research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up to date.
    You want to impress your coworkers with your work.
    """,
    allow_delegation=True,
    vervose = True,
    tools = [
        search_tool,
        scrape_tool
    ],
    max_iter=10,
)

editor = Agent(
    role="Senior Writer/Editor",
    goal="Write engaging blog posts.",
    backstory="""
    You write content that keeps people engaged and entertained.
    Your content is easy to read it is informative and it makes people want to share it with their friends.
    You are working for a very important client.
    """,
    allow_delegation=True,
    vervose=True,
)

# Define tasks
task = Task(
    description="Write a blog post about {topic}",
    agent=editor,
    expected_output="A blog post with an introduction, at least three sub-sections of content, links to sources, a set of suggested hashtags for social media and a catchy title.",
    output_file="blog_post.md",
    output_pydantic = BlogPost
)

crew = Crew(agents=[researcher, editor], tasks=[task], verbose=2)

result = crew.kickoff(
    inputs=dict(
        topic = "The biggest box office flops of 2024",
    ),
)

 [DEBUG]: == Working Agent: Senior Writer/Editor
 [INFO]: == Starting Task: Write a blog post about The biggest box office flops of 2024
 


Search results: Title: Biggest Box Office Bombs Of 2024 - Deadline
Link: https://deadline.com/2025/04/biggest-box-office-bombs-2024-lowest-grossing-movies-1236381446/
Snippet: Joker: Folie à Deux, Megalopolis, Furiosa: A Mad Max Saga, Borderlands and Kraven the Hunter were the biggest box office misses in 2024.
---
Title: Ranked: Biggest Box Office Bombs of 2024 - Visual Capitalist
Link: https://www.visualcapitalist.com/ranked-biggest-box-office-bombs-of-2024/
Snippet: DC's Joker: Folie à Deux was the biggest box office flop of 2024, with a net loss of $144 million. The jukebox musical film is a sequel to the ...
---
Title: What was the biggest Box Office disappointment of 2024? : r/boxoffice
Link: https://www.reddit.com/r/boxoffice/comments/1hrtvu0/what_was_the_biggest_box_office_disappointment_of/
Snippet: Fly me to the moon. The Scarlett Johans

In [5]:
result.introduction

"The film industry is known for its highs and lows, and 2024 was no exception. As audiences flock to theaters for the latest blockbusters, some films simply don't resonate, leading to monumental losses. In this post, we’ll delve into the biggest box office flops of 2024, exploring what went wrong and the lessons learned from these cinematic misfires."